In [ ]:
# Description: This file contains the code for the explainability of the model
# https://towardsdatascience.com/introduction-to-shap-with-python-d27edc23c454
# For binary classification SHAP works with the log-odds of a class.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
import torch
shap.initjs()

from LR_pt import LogisticRegression
from NN import SimpleNN
from data_loader import *

In [ ]:
race_cols = ['Race_American_Indian_Alaska_Native',
        'Race_Asian',
        'Race_Black_African_American',
        'Race_Native_Hawaiian_Pacific_Islander',
        'Race_White',
        'Race_White_Latino']

features = ['loan_amount_000s', 'loan_type', 
       'property_type','applicant_income_000s', 'purchaser_type', 'hud_median_family_income',
       'tract_to_msamd_income', 'number_of_owner_occupied_units', 
       'number_of_1_to_4_family_units', 'race_ethnicity', 'state_code', 'county_code',
       'joint_sex', "minority_population", 'lien_status']

features.remove('race_ethnicity')

clean_features = ['Loan Amount', 'Loan Type', 'Property Type', 'Applicant Income', 'Purchaser Type',
                'Family Income', 'Income Ratio', 'Occupied Units', '1-4 Family Units',
                'State Code', 'County Code', 'Joint Sex', 'Minority Population', 
                'Lien Status']

In [ ]:
data = data_loader(race_cols, num=100000, one_hot=True)
x_train, x_val, x_test, y_train, y_val , y_test, train_groups, val_groups, test_groups = preprocess(data, features, race_cols)
X = torch.from_numpy(x_test).float()
y = torch.from_numpy(np.array(y_test)).long().view(-1, 1)

In [ ]:
# model
model_lr_state = torch.load("models/LRmodel_S:1000000_E:1000_F:True_L:1_G:0.1.pt")
model_lr = LogisticRegression(x_train.shape[1])
model_lr.weights = model_lr_state['linear.weight']
model_lr.bias = model_lr_state['linear.bias']

model_nn_state = torch.load("models/NN_pca:False_E:20_lr:0.001_bs:512.pt")
model_nn = SimpleNN(X.shape[1], 2)
model_nn.load_state_dict(model_nn_state)

In [ ]:
X_np = X.detach().numpy()
explainer = shap.Explainer(model_lr, X_np, feature_names=clean_features)
shap_values_lr = explainer(X_np)

In [ ]:
# Individual instance shap vals
shap.plots.force(shap_values_lr[0])

In [ ]:
## Overall shap vals
# mean SHAP value for each feature
#shap.plots.bar(shap_values_lr[:,:6])
shap.summary_plot(shap_values_lr, X, feature_names=clean_features, plot_type='bar')

# Beeswarm plot
#shap.plots.beeswarm(shap_values_lr[:,:6])
shap.summary_plot(shap_values_lr, X, feature_names=clean_features)

In [ ]:
X_sub = X[:1000]
explainer_nn = shap.DeepExplainer(model_lr, X_sub)
shap_values_nn = explainer_nn.shap_values(X_sub)

In [ ]:
shap.summary_plot(shap_values_nn, X_sub, feature_names=clean_features, plot_type='bar')

In [ ]:
shap.summary_plot(shap_values_nn, X_sub, feature_names=clean_features)